In [3]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, set_seed
import soundfile as sf


e:\practice\talk_assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-expresso").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-expresso")

e:\scrape_multiple_file\02_file\virtual\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
e:\scrape_multiple_file\02_file\virtual\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:
# prompt = '''In the image, there is a man and a woman sitting on grass with their arms around each other. They seem to be embracing or
# cuddling close together, indicating a display of affection or intimacy. The couple appears to be enjoying each other's company in
# what could be described as an outdoor setting, possibly a park or garden.'''
prompt = '''Twinkle, twinkle, little star
How I wonder what you are
Up above the world so high
Like a diamond in the sky
Twinkle, twinkle, little star
How I wonder what you are'''
#description = "Talia in engaging voice and modrate quality audio"
description = "Talia is singing a poem"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

set_seed(42)
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("05.wav", audio_arr, model.config.sampling_rate)

Using the model-agnostic default `max_length` (=2580) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.


In [1]:
import pyaudio
import wave
import torchaudio
import torch

def record_and_save_audio(filename, duration=5):
    """
    Records audio from the default input device, saves it to a WAV file,
    and resamples it to 16000 Hz.

    Args:
        filename (str): Filename to save the WAV file to.
        duration (int): Recording duration in seconds.
    """
    # Open the default input device
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)

    # Record audio
    print("Recording...")
    frames = []
    for i in range(0, int(44100 / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    # Close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded audio to a WAV file
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(2)
    wf.setframerate(44100)
    wf.writeframes(b''.join(frames))
    wf.close()
    print(filename + '.wav'+ 'saved')
    # Load the recorded audio
    #audio_data, sample_rate = torchaudio.load(filename + '.wav')

    # # Resample the audio data to 16000 Hz
    # resampled_audio = torchaudio.transforms.Resample(sample_rate, 16000)(audio_data)

    # # Save the resampled audio data to a new WAV file
    # torchaudio.save(filename + '_resampled.wav', resampled_audio, 16000)
    #print("Recording saved to", filename + '_resampled.wav')

In [5]:
record_and_save_audio('03', duration=10)

Recording...
Recording saved to 03_resampled.wav


In [ ]:
import pyaudio
import wave
import torchaudio
import torch
import tkinter as tk
from threading import Thread

def record_and_save_audio(filename):
    """
    Records audio from the default input device and saves it to a WAV file.

    Args:
        filename (str): Filename to save the WAV file to.
    """
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []

    def stop_recording():
        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(filename + '.wav', 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(2)
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()

        print("Recording saved to", filename + '.wav')

    def start_recording():
        while True:
            data = stream.read(CHUNK)
            frames.append(data)

    recording_thread = Thread(target=start_recording)
    recording_thread.start()

    return stop_recording

def main():
    root = tk.Tk()
    root.title("Audio Recorder")

    def start_recording_callback():
        recording_button.config(state='disabled')
        stop_recording_button.config(state='normal')
        stop_recording = record_and_save_audio('output')
        stop_recording_button.config(command=stop_recording)

    def stop_recording_callback():
        stop_recording_button.config(state='disabled')
        recording_button.config(state='normal')

    recording_button = tk.Button(root, text="Start Recording", command=start_recording_callback)
    recording_button.pack()

    stop_recording_button = tk.Button(root, text="Stop Recording", command=stop_recording_callback, state='disabled')
    stop_recording_button.pack()

    root.mainloop()

if __name__ == "__main__":
    main()

Recording...


: 

In [1]:
5+5

10

In [2]:
5+6

11